# The Local Group Galaxy Database: An Example of a Science Analysis

In [ ]:
import numpy as np
from astropy import units as u
from astropy.coordinates import Distance
from astropy import constants as cnst

from matplotlib import pyplot as plt
%matplotlib inline

from astropy.visualization import quantity_support
quantity_support()

In [ ]:
import galcat

In [ ]:
lgaldb = galcat.Database(directory='data/', references_file='data_references.json')
lgals = lgaldb.query_table()
lgals

In [ ]:
ax = plt.axes(projection='hammer')

msk = lgals['v_mag'] > 0 #bad data  
sc = ax.scatter(lgals['coord'].ra.wrap_at(180*u.deg).rad[msk], lgals['coord'].dec.rad[msk], 
                c=lgals['v_mag'][msk])
plt.colorbar(sc)

As expected, we see the galaxies with a pile of relatively faint ones near M31, and a gap along the Galactic ZOA.  Now lets try making "science plots" a la McConnachie's paper:

In [ ]:
# translate the distance modulus into a true "distance"
lgals['distance'] = Distance(distmod=lgals['distance_modulus'])

msk = ~lgals['radial_velocity'].mask

plt.scatter(lgals['distance'][msk], lgals['radial_velocity'][msk])

What are those odd outliers?  Lets have a look:

In [ ]:
lgals[lgals['radial_velocity']>600]

Arg!  Alternatively masked datasets.  Easy to fix, fortunately:

In [ ]:
msk = (~lgals['radial_velocity'].mask)&(lgals['radial_velocity']<999.9)

plt.scatter(lgals['distance'][msk], lgals['radial_velocity'][msk]*u.km/u.s, alpha=.8)
plt.axvline(300*u.kpc, c='k', ls='--')
plt.axvline(400*u.kpc, c='k', ls=':')
plt.xlim(0, 3*u.Mpc)

Or more physical scaling relations:

In [ ]:
# make the wrong-but-not-horribly wrong assumption of galaxy mass-to-light ratios are 1:
mass_to_light = 1*u.solMass/u.L_sun

vabs = lgals['v_mag'] - lgals['distance'].distmod
mstar = (vabs - 4.83*u.MagUnit(1/u.Lsun)).to(u.L_sun) * mass_to_light

msk = lgals['stellar_radial_velocity_dispersion']!=0

plt.scatter(np.log10(mstar[msk]/u.Msun), lgals['stellar_radial_velocity_dispersion'][msk])
plt.xlabel(f'M* [{mstar.unit.to_string("latex")}]')

What's that upper outlier?

In [ ]:
lgals[lgals['stellar_radial_velocity_dispersion']>80*u.km/u.s]

Oh, that makes sense! It's M32, which is the only "cE" in the Local Group (it's unusually compact and dense).

Maybe instead use a mass estimator like the Wolf et al. 2010 estimator:

In [ ]:
rphys = (lgals['half-light_radius']*lgals['distance']).to(u.kpc, u.dimensionless_angles())
lgals['Mwolf'] = (4 * lgals['stellar_radial_velocity_dispersion']**2 * rphys/ cnst.G).to(u.solMass)

In [ ]:
msk = np.isfinite(lgals['Mwolf'])&(lgals['Mwolf']>0)

plt.scatter(mstar[msk], lgals['Mwolf'][msk])
plt.loglog()
plt.xlabel(f'M* [{mstar.unit.to_string("latex")}]')
plt.ylabel(fr'$M_{{\rm wolf}}$ [{mstar.unit.to_string("latex")}]')

Or we can try the full 3d size-dynamical mass-stellar mass scaling relation:

In [ ]:
from mpl_toolkits import mplot3d
from matplotlib import animation
from IPython import display

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = plt.axes(projection='3d')

msk = np.isfinite(lgals['Mwolf'])&(lgals['Mwolf']>0)

ax.scatter3D(np.log10(rphys[msk]/u.pc),
             np.log10(mstar[msk]/u.solMass),  
             np.log10(lgals['Mwolf'][msk]/u.solMass))

ax.set_xlabel(fr'$\log(r_{{\rm h}}/{u.solMass.to_string("latex_inline")[1:-1]})$', fontsize=18)
ax.set_ylabel(fr'$\log(M_*/{u.solMass.to_string("latex_inline")[1:-1]})$', fontsize=18)
ax.set_zlabel(fr'$\log(M_{{\rm wolf}}/{u.solMass.to_string("latex_inline")[1:-1]})$', fontsize=18)

The "3d" effect isn't great here, so lets try animating it:

In [ ]:
def update_anim(frac):
    ax.azim = frac*360
              
anim = animation.FuncAnimation(fig, update_anim, np.arange(240)/240, interval=1000/30)
display.HTML(anim.to_html5_video())

This yields a set of standard scaling relations I could (for example) use to compare to my favority sets of galaxy formation models.

Hmm, there are some interesting outliers, too.  Let's have a look at what's what:

In [ ]:
for nm in lgals['name'][msk]:
    idx = lgals['name'] == nm
    x = np.log10(rphys[idx]/u.pc)[0]
    y = np.log10(mstar[idx]/u.solMass)[0]
    z = np.log10(lgals['Mwolf'][idx]/u.solMass)[0]
    ax.text3D(x, y, z, nm)

ax.azim = -60
    
fig

What's up with Boötes I down at the bottom of the clump there?  Lets single it out:  

In [ ]:
lgaldb.query_db({'name': 'Bootes (I)'})[0]

The "Stellar Velocity Dispersion" is from Koposov et al. 2011. What's that?:

In [ ]:
lgaldb.query_reference({'key': 'Koposov_2011'})

I look it up and see that's a fine paper... but what if I actually believe the Martin et al. 2007 measurement is the better one? I guess I need to submit it as an option to the DB...

[Go to Github and create/merge the PR]

In [ ]:
lgaldb = galcat.Database(directory='data/', references_file='data_references.json')
lgaldb.query_db({'name': 'Bootes (I)'})[0]

It's in there now.  Now lets try making the radius-dynamical mass plot with the default, and with my preferred reference:

In [ ]:
def make_rM_plot(table, highlight_name=None):
    distance = Distance(distmod=lgals['distance_modulus'])
    rphys = (table['half-light_radius']*distance).to(u.kpc, u.dimensionless_angles())
    Mwolf = (4 * table['stellar_radial_velocity_dispersion']**2 * rphys/ cnst.G).to(u.solMass)
    msk = np.isfinite(Mwolf) & (Mwolf > 0)
    
    if highlight_name is None:
        plt.scatter(rphys[msk], Mwolf[msk])
    else:
        selection = highlight_name == table['name']
        plt.scatter(rphys[msk&~selection], Mwolf[msk&~selection])
        plt.scatter(rphys[msk&selection], Mwolf[msk&selection], c='r')
    plt.loglog()
    
    plt.xlabel(r'$\log(r_{\rm eff}/kpc)$', fontsize=18)
    plt.ylabel(r'$\log(M_{\rm wolf}/\odot)$', fontsize=18)

In [ ]:
make_rM_plot(lgaldb.query_table(selection={'reference': 'Koposov_2011'}), 'Bootes (I)')

In [ ]:
make_rM_plot(lgaldb.query_table(selection={'reference': 'Martin_2007'}), 'Bootes (I)')

In [ ]:
tab = lgaldb.query_table(selection={'reference': 'Martin_2007'})
# V this should not be necssary to get the plot right!
tab['stellar_radial_velocity_dispersion'][tab['name']=='Bootes (I)'] = [6.5]*u.km/u.s
make_rM_plot(tab, 'Bootes (I)')

With that measurement it's now in the middle of the pack.  So maybe my scaling relation is better... Or maybe not!  But I now have the tools to investigate, as a scientist.